# Type 6 Password Encryption for BGP in a Service-Provider Network

You can use Type 6 password encryption to securely store plain text key strings for authenticating BGP, IP SLA, IS-IS, MACsec, OSPF, and RSVP sessions. 

Consider a 4 router SP setup. To authenticate BGP using Type 6 password encryption, you have to enable Type 6 function on each router, and enable BGP on each router. When BGP sessions are in Established state, it indicates that BGP is enabled through Type 6 encryption.
To enable Type 6 password encryption for OSPF, IS-IS, or other protocols, refer the respective configuration guide. For MACsec authentication, refer the Configure MACsec chapter. 

![4-router-topology](4-router-topology-2.png)

## Configuration Steps
* [Set up emulator](#step1)

## <a name="step1"></a>Set up Emulator

>In order to set up the emulator to build our network, you have to execute the cell below (by clicking the play button at the top left part of this notebook.) 

>In the cell below, the python module **Type6_four_routers** sets up the python enviroment and the simulated 4-router test bed along with the base configurations of IPv4 addresses, BGP configurations, Loopback interfaces, etc. After the import, the below code block initialises and brings up the emulator using **sim.init** and **sim.up** API calls.

>The bring up can be slow, 30 minutes +.


In [1]:
from Type6_four_routers import *
sim = Vxr()
sim.no_image_copy=True
sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:v1.1.0 2021-04-23 06:03 output_dir:vxr.out
INFO:pyvxr.vxr:b10e8f6b8c14:/home/vxr/notebooks/Put-Technology-to-Work/BGP-Type6-Password-Encryption
INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
Sim clean: Done
Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/drxsvguisb sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:rp1, rp2, rpe1, rpe2
INFO:pyvxr.console:rp1:wait for XR login prompt (console output captured in vxr.out/logs/console.rp1.log)
INFO:pyvxr.console:rp2:wait for XR login prompt (console output captured in vxr.out/logs/console.rp2.log)
INFO:pyvxr.console:rpe1:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe1.log)
INFO:pyvxr.console:rpe2:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe2.log)
INFO:pyvxr.console:rpe1:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:rpe1:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:rpe1:login successful
INFO:pyvxr.bringup:

Sim status:  {'localhost': 'running'}


>At this point, console access to the router is available. Optionally, you can access the simulated router consoles directly from your laptop  through ssh.

In [2]:
print('Consoles can be reached by:')
print(get_telnet_cmd(sim, 'rpe1'), '\n', get_telnet_cmd(sim, 'rp1'), '\n', get_telnet_cmd(sim, 'rp2'), '\n', get_telnet_cmd(sim, 'rpe2'))
print('or better:')
print(get_ssh_cmd(sim, 'rpe1'), '\n', get_ssh_cmd(sim, 'rp1'), '\n', get_ssh_cmd(sim, 'rp2'), '\n', get_ssh_cmd(sim, 'rpe2'))
print('The password is cisco123')

Consoles can be reached by:
telnet 172.17.0.2 43713 
 telnet 172.17.0.2 35363 
 telnet 172.17.0.2 42706 
 telnet 172.17.0.2 45590
or better:
ssh cisco@172.17.0.2 -p62600 
 ssh cisco@172.17.0.2 -p64708 
 ssh cisco@172.17.0.2 -p62956 
 ssh cisco@172.17.0.2 -p62192
The password is cisco123


>Obtain the telnet addresses to login to the routers.

In [3]:
console_ports = sim.ports()
loginpe1 = telnetlib.Telnet(str(console_ports['rpe1']['HostAgent']) , str(console_ports['rpe1']['serial0']))
loginp1 = telnetlib.Telnet(str(console_ports['rp1']['HostAgent']) , str(console_ports['rp1']['serial0']))
loginp2 = telnetlib.Telnet(str(console_ports['rp2']['HostAgent']) , str(console_ports['rp2']['serial0']))
loginpe2 = telnetlib.Telnet(str(console_ports['rpe2']['HostAgent']) , str(console_ports['rpe2']['serial0']))

BGP sessions

In [4]:
loginpe1.write(('''
show bgp sessions
''').encode('ascii'))
line = loginpe1.read_until(b'/r/n',2)
print(line.decode())


RP/0/RP0/CPU0:PE1#show bgp sessions
Fri Apr 23 06:07:59.268 UTC

Neighbor        VRF                   Spk    AS   InQ  OutQ  NBRState     NSRState
10.1.1.11       default                 0 65537     0     0  Established  None
10.20.20.20     default                 0 65537     0     0  Idle         None
RP/0/RP0/CPU0:PE1#


In [5]:
loginp1.write(('''
show bgp sessions
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print(line.decode())


RP/0/RP0/CPU0:P1#show bgp sessions
Fri Apr 23 06:08:02.919 UTC

Neighbor        VRF                   Spk    AS   InQ  OutQ  NBRState     NSRState
10.1.1.1        default                 0 65537     0     0  Established  None
10.2.2.1        default                 0 65537     0     0  Established  None
10.10.10.10     default                 0 65537     0     0  Idle         None
10.30.30.30     default                 0 65537     0     0  Idle         None
RP/0/RP0/CPU0:P1#


In [6]:
loginp2.write(('''
show bgp sessions
''').encode('ascii'))
line = loginp2.read_until(b'/r/n',2)
print(line.decode())


RP/0/RP0/CPU0:P2#show bgp sessions
Fri Apr 23 06:08:05.813 UTC

Neighbor        VRF                   Spk    AS   InQ  OutQ  NBRState     NSRState
10.2.2.11       default                 0 65537     0     0  Established  None
10.3.3.11       default                 0 65537     0     0  Established  None
10.20.20.20     default                 0 65537     0     0  Idle         None
10.40.40.40     default                 0 65537     0     0  Idle         None
RP/0/RP0/CPU0:P2#


In [7]:
loginpe2.write(('''
show bgp sessions
''').encode('ascii'))
line = loginpe2.read_until(b'/r/n',2)
print(line.decode())


RP/0/RP0/CPU0:PE2#show bgp sessions
Fri Apr 23 06:08:09.027 UTC

Neighbor        VRF                   Spk    AS   InQ  OutQ  NBRState     NSRState
10.3.3.1        default                 0 65537     0     0  Established  None
10.30.30.30     default                 0 65537     0     0  Idle         None
RP/0/RP0/CPU0:PE2#


#### Clean-up the topology

>Once you are done with experimenting on the topology, you should bring down the emulator by executing the following steps:

In [8]:
# close our telnet
loginpe1.close()
loginp1.close()
loginp2.close()

In [9]:
sim.stop()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/drxsvguisb)
INFO:pyvxr.sim:Stopping previous simulation (if any)


In [10]:
sim.clean()
# Clean up sim scratch space
shutil.rmtree(sim_dir)

INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/drxsvguisb)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
